In [1]:
# ftp://ngs.sanger.ac.uk/production/ag1000g/phase1/AR3/variation/crosses/ar3/hdf5/ag1000g.crosses.phase1.ar3sites.3L.h5
# ftp://ngs.sanger.ac.uk/production/ag1000g/phase1/AR3/variation/crosses/ar3/hdf5/ag1000g.crosses.phase1.ar3sites.3R.h5


In [1]:
import pickle
import gzip
import random

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import preprocessing 

%matplotlib inline

In [2]:
mendelian_errors = pickle.load(gzip.open('mendelian_errors.pickle.gz', 'rb'))
feature_fit = np.load(gzip.open('feature_fit.npy.gz', 'rb'))
ordered_features = np.load(open('ordered_features', 'rb'))
num_features = len(ordered_features)

In [3]:
total_observations = len(mendelian_errors)
error_observations = len(list(filter(lambda x: x[0] > 0,mendelian_errors.values())))
ok_observations = total_observations - error_observations
fraction_errors = error_observations/total_observations
print (total_observations, ok_observations, error_observations, 100*fraction_errors)
del mendelian_errors

10905732 10364044 541688 4.9670026734564905


In [5]:
scaler = preprocessing.StandardScaler(copy=False)
scaler.fit(feature_fit[:,:-1])
scaler.transform(feature_fit[:,:-1])
#Original feature fit is now lost
np.save(gzip.open('scaled_fit.npy.gz', 'wb'), feature_fit, allow_pickle=False, fix_imports=False)

In [6]:
feature_fit[:,9].max()

151.27928857724643

In [20]:
feature_fit = np.load(gzip.open('feature_fit.npy.gz', 'rb'))

In [8]:
#total_observations*fraction_errors
prob_ok_choice = error_observations / ok_observations

def accept_entry(row):
    if row[-1] == 1:
        return True
    return random.random() <= prob_ok_choice

accept_entry_v = np.vectorize(accept_entry, signature='(i)->()')

accepted_entries = accept_entry_v(feature_fit)

In [9]:
balanced_fit = feature_fit[accepted_entries]
del feature_fit
balanced_fit.shape
len([x for x in balanced_fit if x[-1] == 1]), len([x for x in balanced_fit if x[-1] == 0])

(541688, 542199)

In [10]:
np.save(gzip.open('balanced_fit.npy.gz', 'wb'), balanced_fit, allow_pickle=False, fix_imports=False)

In [11]:
[balanced_fit[:, column].mean() for column in range(num_features + 1)]

[23297.447150856133,
 353.6483397254511,
 2203.156293967914,
 26.443352529058448,
 0.7400051850423522,
 6.428653959422727,
 48.26200237908218,
 27.39395435132998,
 6.140998517407471,
 6909.07052502565,
 -0.21665604902955368,
 0.499764274320109]

In [12]:
#scaler = preprocessing.StandardScaler(copy=False) #Do not do this
#scaler.fit(balanced_fit[:,:-1])  # especially this!!!!
scaler.transform(balanced_fit[:,:-1])
#Original balanced fit is now lost
scaled_balanced_fit = balanced_fit # Better naming

In [13]:
[scaled_balanced_fit[:, column].mean() for column in range(num_features + 1)]

[-0.11924974054618583,
 0.05537257269568515,
 -0.1261142725981237,
 0.4984897988431588,
 0.07034404738278016,
 0.2103768017770095,
 -0.19192075229384115,
 0.053775154282361515,
 0.9047553788755753,
 0.6011860427175225,
 -0.10151509888827716,
 0.499764274320109]

In [14]:
np.save(gzip.open('scaled_balanced_fit.npy.gz', 'wb'), scaled_balanced_fit, allow_pickle=False, fix_imports=False)

In [15]:
scaled_balanced_fit.shape

(1083887, 12)

In [16]:
cv_indexes = np.random.random(scaled_balanced_fit.shape[0]) > 0.8

In [17]:
scaled_balanced_fit_cv = balanced_fit[cv_indexes]
scaled_balanced_fit_100_test = balanced_fit[~cv_indexes]

In [18]:
np.save(gzip.open('scaled_balanced_fit_cv.npy.gz', 'wb'), scaled_balanced_fit_cv, allow_pickle=False, fix_imports=False)
np.save(gzip.open('scaled_balanced_fit_100_test.npy.gz', 'wb'), scaled_balanced_fit_100_test, allow_pickle=False, fix_imports=False)

In [19]:
np.save(gzip.open('scaled_balanced_fit_10_test.npy.gz', 'wb'), scaled_balanced_fit_100_test[
    np.random.random(scaled_balanced_fit_100_test.shape[0]) <= 0.1], allow_pickle=False, fix_imports=False)
np.save(gzip.open('scaled_balanced_fit_1_test.npy.gz', 'wb'), scaled_balanced_fit_100_test[
    np.random.random(scaled_balanced_fit_100_test.shape[0]) <= 0.01], allow_pickle=False, fix_imports=False)